In [1]:
import logging

import yaml
from pymongo import MongoClient

## Logging

In [2]:
try:
    os.remove('new_ft_sentences.log')
except:
    pass

logging.getLogger().handlers = []
logging.getLogger('requests.packages.urllib3').setLevel(logging.WARNING)
logging.basicConfig(
    filename='new_ft_sentences.log',
    level=logging.INFO,
    format='%(asctime)s %(message)s'
)

## MongoDB

In [3]:
client = MongoClient()
client.drop_database('new_ft_sentences')
DB1 = client.new_ft_text
DB2 = client.new_ft_sentences

In [5]:
def insert_rows(rows):
#     try:
    DB2.rows.insert_many(rows, ordered=False)
#     except Exception as e:
#         write_log('DB INSERTION EXCEPTION', status='{}'.format(e))
#         logging.info('ERROR')

## Search terms

In [1]:
with open('search_terms.yml') as search_term_file:
    term_yaml = yaml.load(search_term_file.read())
    #
    for key in term_yaml:
        d = term_yaml[key]
        for k in d:
            d[k] = list(map(lambda x: '"{}"'.format(x), d[k]))

NameError: name 'yaml' is not defined

## Main

In [7]:
%%time
total = 0
error = 0
path = '/home/antonio/git/nytimes/ft_data/all/'
for e in os.listdir(path):
    f = os.path.join(path, e)
    iterparser = ElementTree.iterparse(f)
    rows = []
    try:
        for event, elem in iterparser:
            if elem.tag == 'item':
                row = parse_item(f, elem)
                rows.append(row)
                elem.clear()
            elif elem.tag == 'ftcom' or elem.tag == 'ftnewspaper':
                elem.clear()
    except ParseError as ex:
        logging.info(e)
        error += 1
    total +=1
    if rows:
            insert_rows(rows)
print('Total files:', total)
print('Error files:', error)

Total files: 281584
Error files: 1318
CPU times: user 6min 23s, sys: 19.2 s, total: 6min 42s
Wall time: 8min 54s


In [8]:
DB.rows.count()

1034401

In [ ]:
%%time
unique = 0
non_unique = 0
for r in DB.rows.find():
    count = DB.rows.find({'article_id': r['article_id']}).count()
    if count == 1:
        unique += 1
    elif count > 1:
        non_unique += 1
    else:
        print(0)
print('Total unique:', unique)
print('Total non unique:', non_unique)

In [11]:
DB.rows.find_one({'article_id': '030102000137'})

{'_id': ObjectId('560e96efa688eb36585c911a'),
 'article_id': '030102000137',
 'pub_date': '20030102',
 'source': 'ftcom',
 'text': 'With people tuned out for the period between Christmas and new year\'s day, it was as good a time as any to sneak out scabrous news. New York\'s senior senator chose instead to send out a traffic warning. The warning itself seemed serious: 8m "unsecured" trucks come in and out of New York every year, posing a "serious terrorist threat". It added: "Over 2,000 shipments of hazardous materials enter and leave New York City by trucks each day." "If we do not secure our truck fleet, this country will still be at grave risk from a terrorist attack," Chuck Schumer said. This is scary indeed - given the number of attacks by fuel-filled trucks on US targets in the past year. But the timing was more than a bit suspect. For one, Schumer\'s wife is Iris Weinshall, who has been commissioner of New York City\'s Department of Transportation for over two years. Could this